In [2]:
#钢材长度对应价格，1m=1,2m=5,3m=8,...,10m=30
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]

In [3]:
#普通dict，当key不在dict中时会报错，而defaultdict不会，会返回0
from collections import defaultdict  

In [4]:
price = defaultdict(int)

In [5]:
for i,p in enumerate(original_price):
    price[i+1] = p

In [6]:
price[10]

30

## Get the max splitting by enumerate

In [7]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30})

求一个10m刚才如何切？  
max(price[10], r[1]+r[9], r[2]+r[8], ... , r[9]+r[1])

如果r[1]+r[9]
r[1] ==> return max(price[1],[])  --> price[1]  
r[9] ==> return max(price[9],[r(1)+r(8), r(2)+r(7),...])
r[8] ==> ...

递归问题需要确定目标结果，比如在该函数中目标结果就是确定r(n)的最佳价格  

两个"+"的不同含义:  
· 第一个list + list,比如[1,2]+[3,4]将两个list拼接，返回[1,2,3,4]   
· 第二个r(i)+r(n-i)，相当于数值相加

In [8]:
def r(n):
    return max([price[n]] + [r(i) + r(n-i) for i in range(1,n)])

In [9]:
r(10)

30

In [10]:
r(15)

43

为什么算r(15)这么慢？  
很多重复的运算...
比如 r(10) -> r(1)+r(9),r(2)+r(8),... -> r(9)=r(1)+r(8)  
每次都要算r(8),而不是第一次算完后直接调出来

In [11]:
import time

### 递归问题

斐波拉契数列：  
1 1 2 3 5...

In [12]:
def fibonacci(n):
    if n  <= 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

In [13]:
start = time.time()
print(fibonacci(34))
end = time.time()
print(end-start)

5702887
3.7840969562530518


In [14]:
mem = defaultdict() #用于存储之前已经算过的
def fibonacci_op(n):
    if n in mem:  #如果在men中，直接调出来使用
        return mem[n]
    else:  #如果不在
        if n <= 2:  
            mem[n] = 1
            return n
        else:
            result = fibonacci_op(n-1) + fibonacci_op(n-2)
            mem[n] = result
            return result

In [15]:
start = time.time()
print(fibonacci_op(64))
end = time.time()
print(end-start)

14662949395604
0.0


# Analysis: How to optimize

## A Simpler Problem

### Decorator

In [16]:
def f1(func):
    def wrapper():
        print('Started')
        func()
        print('Ended')
    return wrapper

In [17]:
def f():
    print('HELLO')

In [18]:
f1(f)()

Started
HELLO
Ended


如何利用装饰器简化上述函数？

In [19]:
@f1
def f():
    print('HELLO')

In [20]:
f()

Started
HELLO
Ended


=> f1(f)()等价于加了装饰器的f()

In [21]:
def f1(func):
    def wrapper(*args,**kwargs):
        print('Started')
        func(*args,**kwargs)
        print('Ended')
    return wrapper

In [22]:
@f1
def f(a):
    print(a)

In [23]:
f('Hello,world')

Started
Hello,world
Ended


函数参数：  
\*args为可变参数  
\*\*kwargs为关键字(key words)参数

In [24]:
def k(*args,**kwargs):
    print(args)  #args以tuple形式
    print(kwargs)  #kwargs以dict形式

In [25]:
k(4,5,6,7,b=7)

(4, 5, 6, 7)
{'b': 7}


In [26]:
def get_time(func):
    def wrapper(*args):
        start = time.time()
        func(*args)
        end = time.time()
        print('used time : {}'.format(end-start))
    return wrapper

In [27]:
@get_time
def f(a):
    print(a)

In [28]:
f('Hello')

Hello
used time : 0.0


In [29]:
print(f.__name__)

wrapper


#### @wrap(view_func)  
消除装饰器对原函数造成的影响  
inference： https://blog.csdn.net/weixin_40576010/article/details/88639686

In [30]:
def decorator(func):
    """this is decorator __doc__"""
    def wrapper(*args, **kwargs):
        """this is wrapper __doc__"""
        print("this is wrapper method")
        return func(*args, **kwargs)
    return wrapper

@decorator
def test():
    """this is test __doc__"""
    print("this is test method")

print("__name__: ", test.__name__)  
print("__doc__:  ", test.__doc__)

__name__:  wrapper
__doc__:   this is wrapper __doc__


test = decorator(test)  
返回wrapper函数，即让test指向了wrapper方法，实际调用了wrapper.\_\_name\_\_,导致查找该方法时本应返回“test”，而返回了装饰器的内置函数。

In [31]:
from functools import wraps
def decorator(func):
    """this is decorator __doc__"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        """this is wrapper __doc__"""
        print("this is wrapper method")
        return func(*args, **kwargs)
    return wrapper

@decorator
def test():
    """this is test __doc__"""
    print("this is test method")

print("__name__: ", test.__name__)  
print("__doc__:  ", test.__doc__)

__name__:  test
__doc__:   this is test __doc__


加入@wraps后，返回了该方法的名字，而不是内置函数。

下面看如何利用装饰器解决我们的切钢材问题...

In [32]:
from functools import wraps

In [33]:
def memo(f):
    memo.already_computed = {}  #存储已经计算过的
    @wraps(f) 
    def _wrap(arg):
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        return result
    return _wrap

# We use this method to solve Cut Rod probelm¶

In [34]:
solution = {}

In [35]:
@memo
def r_op(n):
    """
    Args: n is the iron length
    Return: the max revenue 
    """
    max_price, max_split = max(
        [(price[n], 0)] + [(r_op(i) + r_op(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )

    solution[n] = (n - max_split, max_split)  #记录最佳切的方法
    
    return max_price

In [36]:
s=time.time()
r_op(20)  #大大缩短了计算时间
e=time.time()
print(e-s)

0.0


In [37]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             15: 0,
             14: 0,
             13: 0,
             12: 0,
             11: 0,
             20: 0,
             19: 0,
             18: 0,
             17: 0,
             16: 0})

In [38]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (10, 1),
 12: (10, 2),
 13: (10, 3),
 14: (12, 2),
 15: (13, 2),
 16: (10, 6),
 17: (16, 1),
 18: (16, 2),
 19: (16, 3),
 20: (10, 10)}

# How do we parse solution?¶

In [39]:
def parse_solution(n):
    left_split, right_split = solution[n]
    
    if right_split == 0: return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [41]:
r(17)

48

In [42]:
parse_solution(17)

[10, 6, 1]

# Edit Distance

## 介绍functools.lru_cache

https://blog.csdn.net/wzqnls/article/details/78506022
functools.lru_cache的作用主要是用来做缓存，他能把相对耗时的函数结果进行保存，避免传入相同的参数重复计算。同时，缓存并不会无限增长，不用的缓存会被释放。

In [45]:
from clockdeco import clock
import functools

@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

[0.00000060] fibonacci(0) -> 0
[0.00000090] fibonacci(1) -> 1
[0.00041010] fibonacci(2) -> 1
[0.00000080] fibonacci(1) -> 1
[0.00000070] fibonacci(0) -> 0
[0.00000060] fibonacci(1) -> 1
[0.00006190] fibonacci(2) -> 1
[0.00012450] fibonacci(3) -> 2
[0.00062200] fibonacci(4) -> 3
[0.00000060] fibonacci(1) -> 1
[0.00000060] fibonacci(0) -> 0
[0.00000050] fibonacci(1) -> 1
[0.00005310] fibonacci(2) -> 1
[0.00010570] fibonacci(3) -> 2
[0.00000060] fibonacci(0) -> 0
[0.00000060] fibonacci(1) -> 1
[0.00005780] fibonacci(2) -> 1
[0.00000050] fibonacci(1) -> 1
[0.00000050] fibonacci(0) -> 0
[0.00000060] fibonacci(1) -> 1
[0.00005370] fibonacci(2) -> 1
[0.00010550] fibonacci(3) -> 2
[0.00021520] fibonacci(4) -> 3
[0.00037330] fibonacci(5) -> 5
[0.00105080] fibonacci(6) -> 8
8


In [46]:
from clockdeco import clock

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

[0.00000090] fibonacci(0) -> 0
[0.00000120] fibonacci(1) -> 1
[0.00032880] fibonacci(2) -> 1
[0.00000170] fibonacci(3) -> 2
[0.00040090] fibonacci(4) -> 3
[0.00000160] fibonacci(5) -> 5
[0.00047080] fibonacci(6) -> 8
8


第一段代码中没有加@functools.lru_cache()，有很多重复的计算

## 动态编辑问题

In [47]:
solution = {}

In [48]:
from functools import lru_cache

In [49]:
@lru_cache(maxsize=2**10) #max_size为缓存结果的个数，避免内存爆炸
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2) #s1='',s2='word',删掉s2里的字母
    if len(string2) == 0: return len(string1) #s1='word',s2='',插入s1里的字母
    
    tail_s1 = string1[-1]  #s1='word', tail_s1='d' -> 'wor'+'d'
    tail_s2 = string2[-1]  #s2='work', tail_s2='k' -> 'wor'+'k'
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  #edit_distance('wor','work')  
        # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)), #edit_distance('word','wor')
        # string 1 add tail of string2
    ]
    
    #替换操作
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward) #candidates = [d(DEL),d(INSERT),d(SUB,NOSUB)]
    
    min_distance, operation = min(candidates, key=lambda x: x[0]) #x[0]指根据distance排序
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [50]:
edit_distance('ABCDECG','ABCCEF')

3

In [51]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('ABCDEC', 'A'): 'DEL C',
 ('ABCDEC', 'AB'): 'DEL C',
 ('ABCDEC', 'ABC'): 'DEL C',
 ('ABCDEC', 'ABCC'): '',
 ('ABCDEC', 'ABCCE'): 'DEL C',
 ('ABCDEC', 'ABCCEF'): 'SUB C => F',
 ('ABCDECG', 'A'): 'DEL G',
 ('ABCDECG', 'A

## Todo: Parse Solution is our homework

# Problem Case 3: Pinyin Auto Correction Problem

In [52]:
chinese_dataset = 'article_9k.txt'

In [53]:
CHINESE_CHARATERS = open( chinese_dataset).read()

In [54]:
CHINESE_CHARATERS[:40]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体'

In [57]:
import pinyin

In [58]:
pinyin.get('你好，中国',format='strip',delimiter=' ')

'ni hao ， zhong guo'

In [59]:
#汉字变成拼音
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

In [60]:
CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS)

In [61]:
CHINESE_CHARATERS_COPYS[:100]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi '

In [62]:
len(CHINESE_CHARATERS_COPYS)

129433034

In [64]:
import re

In [65]:
#只提取拼音，不需要数字
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [66]:
tokens(CHINESE_CHARATERS_COPYS[:100])

['ci',
 'wai',
 'zi',
 'ben',
 'zhou',
 'yue',
 'ri',
 'qi',
 'chu',
 'xiao',
 'mi',
 'shou',
 'ji',
 'deng',
 'kuan',
 'ji',
 'xing',
 'wai',
 'qi',
 'yu',
 'ji',
 'xing',
 'yi']

In [67]:
from collections import Counter, defaultdict

In [68]:
#频率次数越多，正确性越高
PINYIN_COUNT = Counter(tokens(CHINESE_CHARATERS_COPYS))

In [69]:
#默认错误拼音的编辑距离最多是2，更多的话无法识别
def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or #known语料库里的拼音集合
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word]) #超过编辑距离为2，无法识别就返回自己
    return max(candidates,key=PINYIN_COUNT.get) #返回语料库里出现最多次的拼音

In [70]:
def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word): #编辑距离为0 = 原词
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits2(word): #编辑距离为2 = 编辑距离1的词上再加一个编辑距离
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [71]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word): #一个词的所有可能拆解方式
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b] #删除[inyin,pnyin,piyin,pinin,pinyn,pinyi]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1] #翻转[ipnyin,]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b] #代替
    inserts = [a+c+b for (a,b) in pairs for c in alphabet] #插入
    return set(deletes + transposes + replaces + inserts)

In [72]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [73]:
print(edits0('pinyin'))

{'pinyin'}


In [74]:
print(edits1('pinyin'))

{'pinyina', 'peinyin', 'oinyin', 'pinypin', 'ainyin', 'mpinyin', 'pinuin', 'pninyin', 'rpinyin', 'pianyin', 'pinyip', 'pzinyin', 'pinyqn', 'pinyrn', 'ponyin', 'pminyin', 'sinyin', 'upinyin', 'pinyxn', 'vpinyin', 'pjinyin', 'pinykin', 'pinyinh', 'apinyin', 'dpinyin', 'pinycin', 'pinyinr', 'pinyinx', 'pifnyin', 'zpinyin', 'winyin', 'pinyino', 'puinyin', 'psnyin', 'pinyit', 'pinyian', 'pinyinm', 'pinyrin', 'pinpyin', 'pynyin', 'pintyin', 'pinyuin', 'pinyikn', 'prnyin', 'pimyin', 'pjnyin', 'pinyis', 'pibnyin', 'pilnyin', 'pinyid', 'piynyin', 'ptinyin', 'pinyi', 'penyin', 'pinytin', 'pinyxin', 'pinyiwn', 'pinyine', 'pinhin', 'pinyqin', 'pinytn', 'pinxyin', 'ninyin', 'piwnyin', 'ypinyin', 'yinyin', 'poinyin', 'pincyin', 'pinyimn', 'pinybn', 'pinyein', 'pinyun', 'pinyipn', 'pinying', 'minyin', 'pinydin', 'pinyiun', 'kpinyin', 'pipnyin', 'pinhyin', 'pniyin', 'piwyin', 'pinlyin', 'pinygn', 'qpinyin', 'pinyins', 'lpinyin', 'pinzyin', 'pinyiq', 'pinuyin', 'pinysin', 'zinyin', 'pinyitn', 'pipyin',

# Test

In [75]:
correct('yin')

'yin'

In [76]:
correct('yign')

'ying'

In [77]:
correct('yinn')

'ying'

In [78]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [79]:
correct_sequence_pinyin('zhe sih yi ge ce sho')

'zhe shi yi ge ce shi'

In [80]:
correct_sequence_pinyin('wo xiang shagn qinng hua da xue')

'wo xiang shang qing hua da xue'

# 思考题-homework？    
#### 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

woyaoshangqinghua
w yaoshangqinghua
wo yaoshangqinghua
woyao shangqinghua

-> DP